Copyright (c) 2020. Cognitive Scale Inc. All rights reserved.
Licensed under CognitiveScale Example Code [License](https://github.com/CognitiveScale/cortex-certifai-examples/blob/7998b8a481fccd467463deb1fc46d19622079b0e/LICENSE.md)

# Introduction

This fourth notebook shows how to scan models for their trust scores using Certifai, using a previously created scan definition as the starting point. 

If you have not already done so, please run the [first notebook](patient-readmission-train.ipynb) to train the models to be explained and the [second notebook](patient-readmission-explain-scan.ipynb) to create the `explain-scan-def.yaml` scan definition.

In this notebook, we will:
1. Load the previously saved `explain-scan-def.yaml` scan definition and models
2. Modify this scan definition to scan for the trust scores (fairness, explainability, robustness)
4. View the results in the Console


In [1]:
import numpy as np
import pandas as pd
import pickle
import pprint

from certifai.scanner.builder import (CertifaiScanBuilder, CertifaiGroupingFeature, 
                                      CertifaiGroupingBucket, CertifaiPredictorWrapper, CertifaiModel, 
                                      CertifaiDataset, CertifaiDatasetSource)

# Loading the Certifai Scan object

In this section, we load the previously defined scan definition to use as a starting point. This is a convenience that avoids us having to recreate information about the prediction task, datasets and feature schema. 

Load the scan definition from file.

In [2]:
scan = CertifaiScanBuilder.from_file('explain-scan-def.yaml')

2021-05-13 07:59:39,701 root   WARNING  The model 'logit' was locally defined and doesn't have a valid 'predict_endpoint'. Before running a scan with this model, you should deploy the model and update the 'logit' models metadata in the scan definition.
2021-05-13 07:59:39,702 root   WARNING  The model 'mlp' was locally defined and doesn't have a valid 'predict_endpoint'. Before running a scan with this model, you should deploy the model and update the 'mlp' models metadata in the scan definition.


Because we're running with local models in the notebook, we need to reload the models and reassociate them with the scan. If the models were running externally, we would instead update the scan definition with the correct predict_endpoint URL.

In [3]:
for model_name in ['logit', 'mlp']:
    with open(f'readmission_{model_name}.pkl', 'rb') as f:
        saved = pickle.load(f)
        model = CertifaiPredictorWrapper(saved.get('model'))
        scan.remove_model(model_name)
        scan.add_model(CertifaiModel(model_name, local_predictor=model))
        

# Create and run the Trust Scan

In this Section, we're now going to modify the scan definition to run a scan for fairness, explainability and robustness. We could include explanations in this new scan, but for now we'll omit them. 

First, remove the explanation evaluation and add fairness, explainability and robustness.

In [4]:
scan.remove_evaluation_type('explanation')
scan.add_evaluation_type('fairness')
scan.add_evaluation_type('robustness')
scan.add_evaluation_type('explainability')

Note that in this analysis, 'fairness' is less about discrimination against protected groups, but can be useful in detecting how specific groups may be more biased to unfavorable predictions. 

To scan for fairness, we need to provide some additional information on the groups to be scanned. We'll choose race, gender and age.

For age, we need to create buckets because it is an integer field with values between 5 and 95

In [5]:
scan.add_fairness_grouping_feature(CertifaiGroupingFeature('race'))
scan.add_fairness_grouping_feature(CertifaiGroupingFeature('gender'))

buckets = []
buckets.append(CertifaiGroupingBucket("05", max=5))
for b in range(10, 91, 10):
    buckets.append(CertifaiGroupingBucket(f"{b}", max=b))
buckets.append(CertifaiGroupingBucket("95"))

scan.add_fairness_grouping_feature(CertifaiGroupingFeature('age', buckets=buckets))

These results need to be interpreted with care.  [Racial/Ethnic Disparities in Readmissions in US Hospitals: The Role of Insurance Coverage](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5946640/) shows that lower readmission rates may not always be construed as a good outcome, and could relate to a lack of insurance coverage and poor access to care. 

Further, for a use case such as this, using an alternative measure such as equalized odds or equal opportunity instead of the default 'burden' based fairness metric may be appropriate. These measures will show where the model has different success rates in prediction across groups and can be measured with Certifai providing ground truth is available. The [fairness metrics notebook](https://github.com/CognitiveScale/cortex-certifai-examples/blob/master/notebooks/fairness_metrics/FairnessMetrics.ipynb) illustrates using alternative metrics.

When running a fairness analysis, it is important that there are sufficient samples of each class within the grouping features. We can check this and other potential issues using a preflight analysis. It will also give us a time estimate for each evaluation.

In [6]:
preflight_result = scan.run_preflight()
pprint.pprint(preflight_result)

Starting Preflight Scan
[--------------------] 2021-05-13 08:01:43.096590 - 0 of 10 checks (0.0% complete) - Running model nondeterminism preflight check for model logit
[##------------------] 2021-05-13 08:01:43.159509 - 1 of 10 checks (10.0% complete) - Running scan time estimate preflight check for model logit


2021-05-13 08:01:43,548 root   WARNING  Bucket 10 of fairness grouping feature age is unrepresented in the data
2021-05-13 08:01:45,158 root   WARNING  Insufficient examples of some fairness classes to guarantee convergence (smallest included class size is for gender='Unknown/Invalid' with 3 samples)


[####----------------] 2021-05-13 08:02:37.704647 - 2 of 10 checks (20.0% complete) - Running one hot checker class preflight check for model logit
[######--------------] 2021-05-13 08:02:37.705070 - 3 of 10 checks (30.0% complete) - Running unknown outcome class preflight check for model logit
[########------------] 2021-05-13 08:02:37.737028 - 4 of 10 checks (40.0% complete) - Running fairness class samples preflight check for model logit
[##########----------] 2021-05-13 08:02:38.318612 - 5 of 10 checks (50.0% complete) - Finished all preflight checks for model logit
[##########----------] 2021-05-13 08:02:38.318837 - 5 of 10 checks (50.0% complete) - Running model nondeterminism preflight check for model mlp
[############--------] 2021-05-13 08:02:38.380013 - 6 of 10 checks (60.0% complete) - Running scan time estimate preflight check for model mlp


2021-05-13 08:02:38,776 root   WARNING  Bucket 10 of fairness grouping feature age is unrepresented in the data
2021-05-13 08:02:40,384 root   WARNING  Insufficient examples of some fairness classes to guarantee convergence (smallest included class size is for gender='Unknown/Invalid' with 3 samples)


[##############------] 2021-05-13 08:03:37.153546 - 7 of 10 checks (70.0% complete) - Running one hot checker class preflight check for model mlp
[################----] 2021-05-13 08:03:37.153961 - 8 of 10 checks (80.0% complete) - Running unknown outcome class preflight check for model mlp
[##################--] 2021-05-13 08:03:37.183810 - 9 of 10 checks (90.0% complete) - Running fairness class samples preflight check for model mlp
[####################] 2021-05-13 08:03:37.770318 - 10 of 10 checks (100.0% complete) - Finished all preflight checks for model mlp
{'logit': {'errors': [],
           'messages': ['Passed model non determinism check',
                        'Expected time for fairness analysis is 388 seconds',
                        'Expected time for robustness analysis is 48 seconds',
                        'Expected time for explainability analysis is 48 '
                        'seconds',
                        'Model logit passed time estimation check',
       

In the warnings, 'gender' has a small sample size (3) for 'Unknown/Invalid'. We will address this by dropping those rows, given they are a tiny proportion and are not a useful class for analysis. 

In other cases (e.g. for underrepresented age ranges), a better approach is to combine smaller classes into one larger class using bucketing. This is illustrated in the [Practical Issues](https://github.com/CognitiveScale/cortex-certifai-examples/blob/master/notebooks/practical_issues/PracticalIssues.ipynb) notebook.

Drop rows with 'gender_Unknown/Invalid' and replace the evaluation dataset in the scan.

In [7]:
df = pd.read_csv('diabetic_data_processed.csv')
scan.remove_dataset('evaluation')
eval_dataset = CertifaiDataset('evaluation',
                                CertifaiDatasetSource.dataframe(df[df['gender_Unknown/Invalid'] == 0]))
scan.add_dataset(eval_dataset)

Save the scan definition and run the scan.

In [8]:
with open('trust-scan-def.yaml', "w") as f:
    scan.save(f)
results = scan.run(write_reports=True)

2021-05-13 08:07:15,995 root   WARNING  The dataset 'evaluation' was loaded from a dataframe and cannot be represented within a scan definition. A default value will be used in the exported YAML for the dataset. Before running a scan with the exported YAML, you should persist the dataset within a file and update the 'evaluation' datasets metadata.
2021-05-13 08:07:15,998 root   WARNING  The model 'logit' was locally defined and cannot be represented within a scan definition because it doesn't have a 'predict_endpoint'. A default value of '<UNKNOWN_ENDPOINT>' will be used in the exported YAML for the models 'predict_endpoint'. Before running a scan with the exported YAML, you should deploy the model and update the 'logit' models metadata in the scan definition.
2021-05-13 08:07:15,999 root   WARNING  The model 'mlp' was locally defined and cannot be represented within a scan definition because it doesn't have a 'predict_endpoint'. A default value of '<UNKNOWN_ENDPOINT>' will be used in 

Starting scan with model_use_case_id: 'readmission' and scan_id: '8738ee6b298a', total estimated time is 19 minutes
[--------------------] 2021-05-13 08:07:19.029479 - 0 of 6 reports (0.0% complete) - Running fairness evaluation for model: logit, estimated time is 388 seconds


2021-05-13 08:07:19,385 root   WARNING  Bucket Unknown/Invalid of fairness grouping feature gender is unrepresented in the data
2021-05-13 08:07:19,434 root   WARNING  Bucket 10 of fairness grouping feature age is unrepresented in the data
2021-05-13 08:09:50,644 root   WARNING  Examples of protected class ('age', '05') exhausted before convergence after 161 samples
2021-05-13 08:20:06,956 root   WARNING  Examples of protected class ('race', 'Asian') exhausted before convergence after 641 samples
2021-05-13 08:21:11,916 root   WARNING  Examples of protected class ('age', '20') exhausted before convergence after 691 samples
2021-05-13 08:36:00,064 root   WARNING  Examples of protected class ('race', 'Other') exhausted before convergence after 1505 samples
2021-05-13 08:39:59,063 root   WARNING  Examples of protected class ('race', 'Hispanic') exhausted before convergence after 2037 samples


[###-----------------] 2021-05-13 08:43:05.583217 - 1 of 6 reports (16.67% complete) - Running robustness evaluation for model: logit, estimated time is 48 seconds
[######--------------] 2021-05-13 08:44:12.087204 - 2 of 6 reports (33.33% complete) - Running explainability evaluation for model: logit, estimated time is 48 seconds
[##########----------] 2021-05-13 08:45:34.975498 - 3 of 6 reports (50.0% complete) - Running fairness evaluation for model: mlp, estimated time is 494 seconds


2021-05-13 08:45:35,451 root   WARNING  Bucket Unknown/Invalid of fairness grouping feature gender is unrepresented in the data
2021-05-13 08:45:35,514 root   WARNING  Bucket 10 of fairness grouping feature age is unrepresented in the data
2021-05-13 08:49:19,685 root   WARNING  Examples of protected class ('age', '05') exhausted before convergence after 161 samples
2021-05-13 09:02:59,019 root   WARNING  Examples of protected class ('race', 'Asian') exhausted before convergence after 641 samples
2021-05-13 09:04:14,235 root   WARNING  Examples of protected class ('age', '20') exhausted before convergence after 690 samples
2021-05-13 09:19:18,745 root   WARNING  Examples of protected class ('race', 'Other') exhausted before convergence after 1505 samples


[#############-------] 2021-05-13 09:21:00.866015 - 4 of 6 reports (66.67% complete) - Running robustness evaluation for model: mlp, estimated time is 58 seconds
[################----] 2021-05-13 09:24:24.680384 - 5 of 6 reports (83.33% complete) - Running explainability evaluation for model: mlp, estimated time is 52 seconds
[####################] 2021-05-13 09:25:52.309533 - 6 of 6 reports (100.0% complete) - Completed all evaluations


# View the Results

The results can be viewed in the Certifai console using the CLI command `certifai console`, run from this folder. 
Go to `http://localhost:8000` in your browser. 

The results can also be analyzed in a notebook. See the [fifth notebook](patient-readmission-trust-results.ipynb) for how to load and work with the results of the trust score scan in a separate notebook.